# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
weather_df = pd.read_csv("../output/cities.csv")
weather_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Price,39.5994,-110.8107,42.80,42,90,9.22
1,1,Tasiilaq,65.6145,-37.6368,30.20,86,75,33.38
2,2,Puerto Ayora,-0.7393,-90.3518,80.85,78,40,3.44
3,3,Bethel,41.3712,-73.4140,49.17,96,100,1.99
4,4,Hilo,19.7297,-155.0900,56.39,88,90,6.91
...,...,...,...,...,...,...,...,...
569,569,Broken Hill,-31.9500,141.4333,55.40,54,99,8.34
570,570,Klyuchi,52.2667,79.1667,54.72,62,80,4.59
571,571,Samandağ,36.0842,35.9771,71.60,43,0,14.97
572,572,Burnie,-41.0667,145.9167,43.47,89,18,3.00


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
#Configure gamps with API key
gmaps.configure(api_key=g_key)

#Use Lat and Lng as location
location = weather_df[["Lat","Lng"]].astype(float)

#Use humidity as weight
humidity = weather_df["Humidity"].astype(float)

In [5]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(location, weights = humidity,
                               dissipating=False, max_intensity=100,
                               point_radius = 1)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
#create a new DF matching given weather critera
#max temp between 70 and 80 degrees
#wind speed less than 10 mph
#zero cloudiness
#drop all rows that don't match above conditions

perfect_cities_df = weather_df.loc[(weather_df["Max Temp"] > 70) & (weather_df["Max Temp"] < 80) &
                                  (weather_df["Wind Speed"] < 10 ) & (weather_df["Cloudiness"] == 0)]

perfect_cities_df.dropna()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
212,212,Lüderitz,-26.6481,15.1594,77.70,40,0,4.88
263,263,Ovalle,-30.5983,-71.2003,76.84,46,0,4.12
454,454,San Miguel de Tucumán,-26.8241,-65.2226,78.80,61,0,2.77
470,470,Abu Samrah,35.3029,37.1841,70.14,38,0,7.47
553,553,Inhambane,-23.8650,35.3833,78.39,69,0,6.91


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
#create df for finding hotels
hotel_df = perfect_cities_df.loc[:,["City", "Lat", "Lng"]]
#add column for hotel name
hotel_df["Hotel Name "] =""
hotel_df

#set base url & parameters for hotel search
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
         "keyword": "hotel",
         "radius" : 5000,
         "key":g_key}

#use for loop to get lat and long for each city and insert into params, once we have full params use same for loop w/ try except to print out search reults
for index, row in hotel_df.iterrows():
    city = row["City"]
    lat = row["Lat"]
    lng = row["Lng"]
    params["locaton"] = f"{lat},{lng}"
    print(f"Searching for Hotels in City #{index}: {city}")
    response = requests.get(base_url, params=params).json()
    hotels = response["results"]


        #try:
            #print(f"The closest hotel in )

Searching for Hotels in City #212: Lüderitz
Searching for Hotels in City #263: Ovalle
Searching for Hotels in City #454: San Miguel de Tucumán
Searching for Hotels in City #470: Abu Samrah
Searching for Hotels in City #553: Inhambane


In [8]:
hotels

[]

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
